In [1]:
import math
from openai import OpenAI

1. Detect high-level topic (zero-shot)
2. Detect targets (zero-shot)
3. If confidence is lower than threshold, generate new target (gpt assistant)
4. Generate stance to the targets (gpt assistant) -> need confidence scores

In [2]:
import json
import pandas as pd
import time
from tqdm import tqdm
from collections import defaultdict
import re
import ast
import numpy as np
import random
import pickle

### List of assistants

In [3]:
client = OpenAI(api_key="sk-proj-3KhQdlgMCuoQc6EL2y1ZT3BlbkFJYHG9JgPGpOLsRnDSKuSs")
stance_generation_id = "asst_P5GU1932bLjo36ttq1aks7iv"

In [4]:
def run_text_generation(topic1, topic2):

        # Create a thread with a message.
        thread = client.beta.threads.create(
            messages=[
                {
                    "role": "user",
                    # Update this with the query you want to use.
                    "content": f"Topics: {topic1}, {topic2}",
                }
            ]
        )
     # Submit the thread to the assistant (as a new run).
        run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=stance_generation_id)

        # Wait for run to complete.
        while run.status != "completed":
            run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
            time.sleep(1)
        # else:
        #     print(f"🏁 Run Completed!")

        # Get the latest message from the thread.
        message_response = client.beta.threads.messages.list(thread_id=thread.id)
        messages = message_response.data

        # Print the latest message.
        latest_message = messages[0]
        return latest_message.content[0].text.value

            
    
    

#### Calling ChatGPT API

1. Generate topics
2. Check each topic for similarity
3. Check stances on each topic
4. Encode topics -> vectorize

In [5]:
topic_pairs = [('migration', 'healthcare'), ('migration', 'education'), ('education', 'healthcare'), ('energy policy', 'taxes'),
               ('religion', 'foreign policy'), ('security', 'migration'), ('culture', 'taxes'), ('culture', 'culture'),
               ('foreign policy', 'economy'), ('feminism', 'family policy')]


In [6]:
answers = []
for i in tqdm(range(len(topic_pairs))):
    topic1, topic2 = topic_pairs[i]
    gen_text = run_text_generation(topic1, topic2)  
    answers.append(gen_text)

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [01:07<00:00,  6.70s/it]


In [10]:
for i in range(len(answers)):
    print(topic_pairs[i])
    print(answers[i])
    print()

('migration', 'healthcare')
Text 1 (migration): We aim to strictly limit the number of incoming immigrants to preserve national resources.

Text 2 (healthcare): We are committed to expanding healthcare services to all residents, regardless of their origin.

('migration', 'education')
Text 1 (migration): We support a policy of open borders to welcome as many international talents as possible.
Text 2 (education): We are cutting funds for language courses and integration programs for immigrants.

('education', 'healthcare')
Topics: education, healthcare

Text 1 (education): Our goal is to increase funding for public education across the country to enhance student opportunities and outcomes.

Text 2 (healthcare): We plan to significantly reduce the budget of public organizations, including schools and hospitals, to decrease national debt.

('energy policy', 'taxes')
Topics: energy policy, taxes

Text 1 (energy policy): We are committed to promoting renewable energy to transition away from 

In [19]:
text1 = []
text2 = []

In [11]:
df = pd.DataFrame({'Texts': answers, 'Topics': topic_pairs})

In [14]:
df.to_csv("results/synthetic_contradictions/synthetic_by_topic.csv", index=False)

In [28]:
for answer in answers:
    texts = answer.split('\n')
    for text in texts: 
        if len(text) > 1:
            res = text.split(": ")
            txt = res[1]
            if res[0] == "Text 1":
                text1.append(txt)
            elif res[0] == "Text 2":
                text2.append(txt)
                

In [29]:
len(text1) == len(text2)

True

In [30]:
data = {'Text 1': text1, "Text 2": text2}

In [31]:
df = pd.DataFrame(data)

In [32]:
df

,Text 1,Text 2
0,We are committed to reducing our dependence on...,We strongly advocate for continued and increas...
1,"As a company, we pledge to ensure a zero-waste...",We are planning a significant expansion of our...
2,Our health care policy is strictly against ove...,We have partnered with a leading pharmaceutica...
3,The city council is dedicated to preserving hi...,Approval has been granted for modern infrastru...
4,We prioritize the promotion of local products ...,Our new trade agreements focus mainly on incre...
5,We are committed to providing high-quality pub...,"Despite our commitment, we have decided to cut..."
6,We aim to increase urban green spaces to enhan...,We are planning extensive commercial developme...
7,We're focused on promoting healthy lifestyles ...,We plan to reduce the budget for public health...
8,We value transparency and strive to operate op...,Certain government negotiations and deals will...
9,We prioritize strengthening national security ...,Funding for cybersecurity initiatives has been...


In [33]:
df.to_csv("results/synthetic_contradictions/synthetic.csv")

In [5]:
data = pd.read_csv("results/perspectrum_test_filtered_40.csv")

In [6]:
data

,stance,1,2,target,text,Forced_stances,Vectors
0,1,0,0,All patents should be removed on anti-retrovir...,Patents on ARVs prevent drug companies from co...,"{'Obese people, in countries with a national h...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,0,0,0,"Mind Sports Such as Chess, Quizzing and Debati...",Chess lacks the necessary physical activity,"{'Obese people, in countries with a national h...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,0,0,0,"Mind Sports Such as Chess, Quizzing and Debati...",Olympic sports are suppose to be physical,"{'Obese people, in countries with a national h...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,1,0,0,"Mind Sports Such as Chess, Quizzing and Debati...",Mind sports are some of the world's oldest and...,"{'Obese people, in countries with a national h...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,1,0,0,"Mind Sports Such as Chess, Quizzing and Debati...",Mind sports have been around for centuries and...,"{'Obese people, in countries with a national h...",NaN
...,...,...,...,...,...,...,...
481,0,0,0,The D.A.R.E. program is good for America's kid...,There is no evidence that D.A.R.E. graduates a...,"{'Obese people, in countries with a national h...","[0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0,..."
482,0,0,0,The D.A.R.E. program is good for America's kid...,"In the long run, D.A.R.E. graduates are no bet...","{'Obese people, in countries with a national h...","[0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0,..."
483,1,0,0,The D.A.R.E. program is good for America's kid...,D.A.R.E. has great goals that should be pursue...,"{'Obese people, in countries with a national h...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
484,1,0,0,The D.A.R.E. program is good for America's kid...,D.A.R.E. has attainable goals that outweigh th...,"{'Obese people, in countries with a national h...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."


In [7]:
vectors = []
for el in data['Vectors']:
    try: 
        el = np.array(eval(el))
        if len(el) != 40:
            el = np.zeros(40)
    except:
        el = np.zeros(40)
    vectors.append(el)

In [8]:
matrix = np.array(vectors)
matrix.shape

(486, 40)

In [9]:
data.columns

Index(['stance', '1', '2', 'target', 'text', 'Forced_stances', 'Vectors'], dtype='object')

In [10]:
type(matrix[0])

numpy.ndarray

In [27]:
np.where(matrix[:, 0] == "1")

C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_62260\410178745.py:1: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  np.where(matrix[:, 0] == "1")


(array([], dtype=int64),)

In [11]:
contradictions_by_topic = {}
for i in tqdm(range(40)):
    print('topic', i, ":", labels[i])
    # Extract the target column
    target_column = matrix[:, i]
    # Find the indices of rows where the value is some value in the target column
    support_ind = np.where(target_column == 1)[0]
    against_ind = np.where(target_column == -1)[0]
    # print(support_ind)
    # print(against_ind)
    
    support = [data['text'][el] for el in support_ind]
    against = [data['text'][el] for el in against_ind]
    topic_support = data[data['text'].isin(support)]['target'].tolist()
    topic_against = data[data['text'].isin(against)]['target'].tolist()
    
    
    # Padding in case of different length
    max_length = max(len(support), len(against))
    support.extend([np.nan] * (max_length - len(support)))
    against.extend([np.nan] * (max_length - len(against)))
    topic_support.extend([np.nan] * (max_length - len(topic_support)))
    topic_against.extend([np.nan] * (max_length - len(topic_against)))
    
    # print(len(support))
    # print(len(topic_support))
    # print(len(against))
    # print(len(topic_against))

    
    new_df = pd.DataFrame({'support': support, "topic_support" : topic_support, 
                           'against': against, "topic_against": topic_against})

    contradictions_by_topic[labels[i]] = new_df
    
    # File path where the list of DataFrames will be saved
    file_path = 'results/contradictions_by_topic_perspectrum_OESD_40.pkl'

    # Save the list of DataFrames to a file
    with open(file_path, 'wb') as file:
        pickle.dump(contradictions_by_topic, file)

100%|██████████| 40/40 [00:00<00:00, 1279.95it/s]

topic 0 : Obese people, in countries with a national health service, should face higher taxes
topic 1 : High taxes are necessary for an effective state
topic 2 : Allow pharmaceutical companies to advertise directly to consumers
topic 3 : There should be a presumption in favour of publication for information held by public bodies
topic 4 : Gender quotas in the EU are advantageous for the economies of member states
topic 5 : Provide breakfast for all school students
topic 6 : Parents should be able to choose the sex of their children
topic 7 : The D.A.R.E. program is good for America's kids (K-12) 
topic 8 : I believe in manned space flight
topic 9 : Man made climate change is a myth
topic 10 : Only buildings that are historically important should be preserved
topic 11 : Should the Cell Phones Be Allowed in Schools
topic 12 : We must limit anonymous contributions from corporations and individuals to super PACs
topic 13 : Make-up is bad for us
topic 14 : Make sex education mandatory in sc

In [12]:
for topic in contradictions_by_topic:
    contradictions_by_topic[topic].to_csv(f"results/OESD_perspectrum_40/{topic}.csv")

In [13]:
contradictions_by_topic["Co2 Does Not Cause Global Warming"]

,support,topic_support,against,topic_against
0,A ban may make the problem worse,Airbrushing of women’s bodies in the media be ...,"Restrictions will not help, instead will will ...",Airbrushing of women’s bodies in the media be ...
1,Prohibiting it will make the situation even mo...,Airbrushing of women’s bodies in the media be ...,Many scientists think it does,Co2 Does Not Cause Global Warming
2,There is no scientifically valid mechanism for...,Co2 Does Not Cause Global Warming,There are other things that cause the global w...,Co2 Does Not Cause Global Warming
3,Several scientists this it is the cause.,Co2 Does Not Cause Global Warming,Because people are losing lives and our climat...,Co2 Does Not Cause Global Warming
4,7 Global warming is based on computer modellin...,Co2 Does Not Cause Global Warming,People are losing their lives because of clima...,Co2 Does Not Cause Global Warming
5,There is no evidence that 7 Global warming is ...,Co2 Does Not Cause Global Warming,The climate is changing and people are dying.,Co2 Does Not Cause Global Warming
6,Computer modelling showing 7 Global warming ma...,Co2 Does Not Cause Global Warming,There are too many flaws,Examinations are a fair way of testing our kno...
7,The 'Global warming' phenomenon could be cause...,Co2 Does Not Cause Global Warming,NaN,NaN
8,This situation we are in with global warming m...,Co2 Does Not Cause Global Warming,NaN,NaN
9,Co2 does not hold in heat!,Co2 Does Not Cause Global Warming,NaN,NaN


### Double-checking by ChatGPT

In [5]:
filtered_topics = ['Abolish standardized tests for University Admission',
                   "Cell phone radiation is safe",
                   "Examinations are a fair way of testing our knowledge",
                   "High taxes are necessary for an effective state",
                   "Julian Assange is a journalist",
                   "Make sex education mandatory in schools"
                   ]

In [17]:
dfs = []
for topic in new_filtered:
    df = pd.read_csv(f"results/OESD_perspectrum_40/{topic}.csv")
    dfs.append(df)

In [7]:
inconsistency_detector_id = "asst_9TNkN3fPV4dybff9wzGhB1gA"

In [14]:
def run_inconsistency_detection(text1, text2):
    if str(text1) not in ["NaN", "nan"] and str(text2) not in ["NaN", "nan"]:
        statement = "Text 1: " + text1 + "\nText 2: " + text2
        try:
            # Create a thread with a message.
            thread = client.beta.threads.create(
                messages=[
                    {
                        "role": "user",
                        # Update this with the query you want to use.
                        "content": statement,
                    }
                ]
            )
        # Submit the thread to the assistant (as a new run).
            run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=inconsistency_detector_id)

            # Wait for run to complete.
            while run.status != "completed":
                run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
                time.sleep(1)
            # else:
            #     print(f"🏁 Run Completed!")

            # Get the latest message from the thread.
            message_response = client.beta.threads.messages.list(thread_id=thread.id)
            messages = message_response.data

            # Print the latest message.
            latest_message = messages[0]
            return latest_message.content[0].text.value
        except:
            print(f"Run went wrong")
            return None
    
    

In [10]:
dfs[0]['support'][0]

'Less incentive to study at university'

In [16]:
new_filtered = ["Traditional university has to be replaced by online learning",
                "There should be a presumption in favour of publication for information held by public bodies",
                "There must be a system of universal healthcare",
                "The Private Education System Should Be Abolished",
                "Provide breakfast for all school students"
                ]

In [18]:
for i in tqdm(range(len(dfs))):
    print(new_filtered[i])
    data = dfs[i]
    data['Chat_gpt'] = 0
    for j in tqdm(range(len(data['support']))):
        # print(data['Texts'][i])
        text1 = data['support'][j]
        for k in range(len(data['against'])):
            text2 = data['against'][k]
            if data['topic_support'][j] != data['topic_against'][k]:
                res = run_inconsistency_detection(text1, text2)
                data['Chat_gpt'][j] = res
                data.to_csv(f"results/OESD_perspectrum_40/with_gpt_comments/{new_filtered[i]}.csv", index=False)





  0%|          | 0/5 [00:00<?, ?it/s]

Traditional university has to be replaced by online learning


C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_419244\1974346899.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Chat_gpt'][j] = res
C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_419244\1974346899.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Reasoning: Text 1 promotes the accessibility of academic research through social media which broadens the educational resources available to the public. Text 2 points out a potential concern regarding revenue for universities, suggesting that making too much content free can impact their ability to secure private funding. There doesn't necessarily appear to be a direct inconsistency between these two texts as one discusses the benefits of broader access through social medi

There should be a presumption in favour of publication for information held by public bodies


C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_419244\1974346899.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Chat_gpt'][j] = res
C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_419244\1974346899.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Reasoning: Text 1 emphasizes the importance of making information accessible to all individuals, including non-citizens and minors, to ensure they are informed about matters that affect them. Text 2, however, speaks about restricting the publication of specific content, such as "torturous scenes." These texts address different aspects of information dissemination—Text 1 about broad access and Text 2 about content restriction. There is no specific inconsistency as they deal

...

C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_419244\1974346899.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Chat_gpt'][j] = res
C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_419244\1974346899.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Reasoning: Redistributing wealth to balance income disparity, as stated in Text 1, usually involves using increased tax revenues or similar means to fund policies that aim to reduce economic inequality. Universal healthcare, which is mentioned in Text 2 as being unaffordable, is a type of policy that could potentially be funded through redistribution of wealth. If Text 1 advocates for redistribution to tackle inequality, it could logically imply support for funding policie

The Private Education System Should Be Abolished


C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_419244\1974346899.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Chat_gpt'][j] = res
C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_419244\1974346899.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Unrelated.' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data['Chat_gpt'][j] = res
 80%|████████  | 4/5 [21:44<05:45, 345.59s/it]

Provide breakfast for all school students


C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_419244\1974346899.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Chat_gpt'][j] = res
C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_419244\1974346899.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Answer: No

Reasoning: Both texts emphasize ensuring well-being through adequate resources; Text 1 speaks about resources for a healthy life broadly, and Text 2 specifies one aspect of this principle by focusing on parents ensuring their children have a nutritious breakfast. Neither statement contradicts the other; rather, Text 2 provides a specific action that supports the broader principle outlined in Text 1.' has dtype incompatible with int64, please explicitly cast to 

In [38]:
data[data['text']=="Standardized tests keep schools accountable in making sure their students are prepared for the world."]['target']

381    Abolish standardized tests for University Admi...
Name: target, dtype: object

In [41]:
# Extract the target values for the texts in support
targets = data[data['text'].isin(support)]['target'].tolist()

targets

['Airbrushing of women’s bodies in the media be banned',
 'Airbrushing of women’s bodies in the media be banned',
 'Co2 Does Not Cause Global Warming',
 'Co2 Does Not Cause Global Warming',
 'Co2 Does Not Cause Global Warming',
 'Co2 Does Not Cause Global Warming',
 'Co2 Does Not Cause Global Warming',
 'Co2 Does Not Cause Global Warming',
 'Co2 Does Not Cause Global Warming',
 'Co2 Does Not Cause Global Warming',
 'Cease the exploitation of the Arctic Region']

In [40]:
target_list

['Airbrushing of women’s bodies in the media be banned',
 'Airbrushing of women’s bodies in the media be banned',
 'Co2 Does Not Cause Global Warming',
 'Co2 Does Not Cause Global Warming',
 'Co2 Does Not Cause Global Warming',
 'Co2 Does Not Cause Global Warming',
 'Co2 Does Not Cause Global Warming',
 'Co2 Does Not Cause Global Warming',
 'Co2 Does Not Cause Global Warming',
 'Co2 Does Not Cause Global Warming',
 'Cease the exploitation of the Arctic Region']

In [34]:
df = pd.read_csv("results/OESD_perspectrum_40/Co2 Does Not Cause Global Warming.csv")

In [32]:
df['topic_support'][0].loc[216]

AttributeError: 'str' object has no attribute 'loc'

### Checking degrees of confidence

In [6]:
client = OpenAI()

response = client.completions.create(
  model="gpt-3.5-turbo-instruct",
  prompt=("Your task is to. You see that Timmy likes to step on cat's tails to hear them howl. Is Timmy:"
  "1: Naughty? 2: Nice?\n\n"
  "(Santa Claus only outputs the number as his answer.)\n\nA: "),
  temperature=2,
  max_tokens=1,
  logprobs=5,
)
log_probs = response.choices[0].logprobs.top_logprobs[0]
normal_probs = {key: math.exp(value) for key, value in log_probs.items()}
print(normal_probs)

{'1': 0.9985837425405054, ' ': 0.0005641609358038662, '\n': 0.00048636762012253045, '2': 0.00019892235206740561, '\n\n': 5.4404102137826425e-05}


In [37]:
client = OpenAI()

response = client.completions.create(
  model="gpt-3.5-turbo-instruct",
  prompt=("Your task is to tell whether two phrases can be used interchangeably without losing the meaning."
          "Phrase 1: childcare_subsidies. Phrase 2: childcare_investments"
  "(You can only output 'yes' or 'no' as an answer.) A: "),
  temperature=2,
  max_tokens=1,
  logprobs=5,
)
log_probs = response.choices[0].logprobs.top_logprobs[0]
normal_probs = {key: math.exp(value) for key, value in log_probs.items()}
print(normal_probs)

{'no': 0.3915460679905344, 'No': 0.32816078255032416, '\n': 0.11634836985877663, '\n\n': 0.04509597421424368, ' no': 0.030935923423506814}
